In [1]:
import pandas as pd
import numpy as np

In [2]:
from matplotlib import pyplot as plt

In [3]:
us_time_series = pd.read_csv('covid_dataset/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
us_time_series = us_time_series.drop(['UID','iso2','iso3','code3','FIPS','Admin2','Country_Region','Lat','Long_', 'Combined_Key'],axis=1)

In [4]:
# us_time_series.head()

In [5]:
us_time_series = us_time_series.drop(us_time_series.columns[[*range(1,50)]],axis=1)
# us_time_series.head()

In [6]:
nc_time_series = us_time_series[us_time_series['Province_State']=='North Carolina']

list_err = []

for i in range(nc_time_series.shape[0]):

    case_data = nc_time_series.iloc[0,1:]

    case_date = list(nc_time_series.columns)[1:]

    data = pd.DataFrame(zip(case_date, case_data), columns=['dates', '#cases'])

    from datetime import datetime
    con=data['dates']
    data['dates']=pd.to_datetime(data['dates'])
    data.set_index('dates', inplace=True)
    #check datatype of index
    data.index

    #convert to time series:
    ts = data['#cases']
    ts.head(10)

    # plt.plot(ts)

    from statsmodels.tsa.stattools import adfuller
    def test_stationarity(timeseries):

        #Determing rolling statistics
        rolmean = timeseries.rolling(12).mean()
        rolstd = timeseries.rolling(12).std()
    #Plot rolling statistics:
        # plt.plot(timeseries, color='blue',label='Original')
        # plt.plot(rolmean, color='red', label='Rolling Mean')
        # plt.plot(rolstd, color='black', label = 'Rolling Std')
        # plt.legend(loc='best')
        # plt.title('Rolling Mean & Standard Deviation')
        # plt.show()
        #Perform Dickey-Fuller test:
        # print('Results of Dickey-Fuller Test:')
        dftest = adfuller(timeseries, autolag='AIC')
        dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
        for key,value in dftest[4].items():
            dfoutput['Critical Value (%s)'%key] = value
        # print(dfoutput)

    test_stationarity(ts)

    ts_log = np.log(ts)
    # plt.plot(ts_log)

    ts_log.replace([np.inf, -np.inf], np.nan, inplace=True)
    ts_log.dropna(inplace=True)

    moving_avg = ts_log.rolling(12).mean()
    # plt.plot(ts_log)
    # plt.plot(moving_avg, color='r')

    ts_log_moving_avg_diff = ts_log - moving_avg
    ts_log_moving_avg_diff.head(12)

    ts_log_moving_avg_diff.dropna(inplace=True)
    ts_log_moving_avg_diff.head()

    test_stationarity(ts_log_moving_avg_diff)

    ts_log_diff = ts_log - ts_log.shift()
    # plt.plot(ts_log_diff)

    # ts_log_diff.dropna(inplace=True)
    # test_stationarity(ts_log_diff)

    from statsmodels.tsa.arima.model import ARIMA
    from statsmodels.tsa.stattools import acf, pacf

    lag_acf = acf(ts_log, nlags=20)
    lag_pacf = pacf(ts_log, nlags=20, method='ols')

    model = ARIMA(ts_log_diff, order = (1,1,1))

    results = model.fit()

    # plt.plot(ts_log_diff)
    # plt.plot(results.fittedvalues, color='r')

    sum_result = results.summary()
    sum_result

    # Plot residual errors
    residuals = pd.DataFrame(results.resid)
    fig, ax = plt.subplots(1,2)
    residuals.plot(title="Residuals", ax=ax[0])
    residuals.plot(kind='kde', title='Density', ax=ax[1])
    # plt.show()

    results.pvalues

    err = results.standardized_forecasts_error.mean()
    list_err.append(err)

In [9]:
nc_err = np.array(list_err)

In [11]:
sc_time_series = us_time_series[us_time_series['Province_State']=='South Carolina']

list_err = []

for i in range(sc_time_series.shape[0]):

    case_data = sc_time_series.iloc[0,1:]

    case_date = list(sc_time_series.columns)[1:]

    data = pd.DataFrame(zip(case_date, case_data), columns=['dates', '#cases'])

    from datetime import datetime
    con=data['dates']
    data['dates']=pd.to_datetime(data['dates'])
    data.set_index('dates', inplace=True)
    #check datatype of index
    data.index

    #convert to time series:
    ts = data['#cases']
    ts.head(10)

    # plt.plot(ts)

    from statsmodels.tsa.stattools import adfuller
    def test_stationarity(timeseries):

        #Determing rolling statistics
        rolmean = timeseries.rolling(12).mean()
        rolstd = timeseries.rolling(12).std()
    #Plot rolling statistics:
        # plt.plot(timeseries, color='blue',label='Original')
        # plt.plot(rolmean, color='red', label='Rolling Mean')
        # plt.plot(rolstd, color='black', label = 'Rolling Std')
        # plt.legend(loc='best')
        # plt.title('Rolling Mean & Standard Deviation')
        # plt.show()
        #Perform Dickey-Fuller test:
        # print('Results of Dickey-Fuller Test:')
        dftest = adfuller(timeseries, autolag='AIC')
        dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
        for key,value in dftest[4].items():
            dfoutput['Critical Value (%s)'%key] = value
        # print(dfoutput)

    test_stationarity(ts)

    ts_log = np.log(ts)
    # plt.plot(ts_log)

    ts_log.replace([np.inf, -np.inf], np.nan, inplace=True)
    ts_log.dropna(inplace=True)

    moving_avg = ts_log.rolling(12).mean()
    # plt.plot(ts_log)
    # plt.plot(moving_avg, color='r')

    ts_log_moving_avg_diff = ts_log - moving_avg
    ts_log_moving_avg_diff.head(12)

    ts_log_moving_avg_diff.dropna(inplace=True)
    ts_log_moving_avg_diff.head()

    test_stationarity(ts_log_moving_avg_diff)

    ts_log_diff = ts_log - ts_log.shift()
    # plt.plot(ts_log_diff)

    # ts_log_diff.dropna(inplace=True)
    # test_stationarity(ts_log_diff)

    from statsmodels.tsa.arima.model import ARIMA
    from statsmodels.tsa.stattools import acf, pacf

    lag_acf = acf(ts_log, nlags=20)
    lag_pacf = pacf(ts_log, nlags=20, method='ols')

    model = ARIMA(ts_log_diff, order = (1,1,1))

    results = model.fit()

    # plt.plot(ts_log_diff)
    # plt.plot(results.fittedvalues, color='r')

    sum_result = results.summary()
    sum_result

    # Plot residual errors
    residuals = pd.DataFrame(results.resid)
    fig, ax = plt.subplots(1,2)
    residuals.plot(title="Residuals", ax=ax[0])
    residuals.plot(kind='kde', title='Density', ax=ax[1])
    # plt.show()

    results.pvalues

    err = results.standardized_forecasts_error.mean()
    list_err.append(err)

In [13]:
sc_err = np.array(list_err)

In [15]:
from scipy import stats

In [23]:
res = stats.ttest_ind(nc_err, sc_err, equal_var=False)

In [24]:
res

Ttest_indResult(statistic=-4289933547973839.5, pvalue=0.0)

In [25]:
nc_err

array([-0.06230687, -0.06230687, -0.06230687, -0.06230687, -0.06230687,
       -0.06230687, -0.06230687, -0.06230687, -0.06230687, -0.06230687,
       -0.06230687, -0.06230687, -0.06230687, -0.06230687, -0.06230687,
       -0.06230687, -0.06230687, -0.06230687, -0.06230687, -0.06230687,
       -0.06230687, -0.06230687, -0.06230687, -0.06230687, -0.06230687,
       -0.06230687, -0.06230687, -0.06230687, -0.06230687, -0.06230687,
       -0.06230687, -0.06230687, -0.06230687, -0.06230687, -0.06230687,
       -0.06230687, -0.06230687, -0.06230687, -0.06230687, -0.06230687,
       -0.06230687, -0.06230687, -0.06230687, -0.06230687, -0.06230687,
       -0.06230687, -0.06230687, -0.06230687, -0.06230687, -0.06230687,
       -0.06230687, -0.06230687, -0.06230687, -0.06230687, -0.06230687,
       -0.06230687, -0.06230687, -0.06230687, -0.06230687, -0.06230687,
       -0.06230687, -0.06230687, -0.06230687, -0.06230687, -0.06230687,
       -0.06230687, -0.06230687, -0.06230687, -0.06230687, -0.06